# Phenotype

lamindb provides access to the following public protein ontologies through [lnschema-bionty](https://github.com/laminlabs/lnschema-bionty):

1. [Human Phenotype](https://hpo.jax.org/app)
2. [Phecodes](https://phewascatalog.org/phecodes_icd10)
3. [PATO](https://github.com/pato-ontology/pato)
4. [Mammalian Phenotype](http://obofoundry.org/ontology/mp.html)

Here we show how to access and search Phenotype ontologies to standardize new data.

## Setup

In [ ]:
!lamin init --storage ./test-phenotype --schema bionty

In [ ]:
import lnschema_bionty as lb
import pandas as pd

## PublicOntology objects

Let us create a public ontology accessor with {meth}`~docs:lnschema_bionty.dev.BioRegistry.public`, which chooses a default public ontology source from {class}`~docs:lnschema_bionty.PublicSource`.
It's a [PublicOntology](https://lamin.ai/docs/lnschema_bionty.dev.publicontology) object, which you can think about as a public registry:

In [ ]:
phenotypes = lb.Phenotype.public(organism="human")
phenotypes

As for registries, you can export the ontology as a `DataFrame`:

In [ ]:
df = phenotypes.df()
df.head()

Unlike registries, you can also export it as a Pronto object via `public.ontology`.

## Look up terms

As for registries, terms can be looked up with auto-complete:

In [ ]:
lookup = phenotypes.lookup()

The `.` accessor provides normalized terms (lower case, only contains alphanumeric characters and underscores):

In [ ]:
lookup.eeg_with_persistent_abnormal_rhythmic_activity

To look up the exact original strings, convert the lookup object to dict and use the `[]` accessor:

In [ ]:
lookup_dict = lookup.dict()
lookup_dict["EEG with persistent abnormal rhythmic activity"]

By default, the `name` field is used to generate lookup keys. You can specify another field to look up:

In [ ]:
lookup = phenotypes.lookup(phenotypes.ontology_id)

In [ ]:
lookup.hp_0000003

## Search terms

Search behaves in the same way as it does for registries:

In [ ]:
phenotypes.search("dysplasia").head(3)

By default, search also covers synonyms:

In [ ]:
phenotypes.search("Congenital hip dysplasia").head(3)

You can turn this off synonym by passing `synonyms_field=None`:

In [ ]:
phenotypes.search("Congenital hip dysplasia", synonyms_field=None).head(3)

Search another field (default is `.name`):

In [ ]:
phenotypes.search(
    "lack of development of speech and language",
    field=phenotypes.definition,
).head()

## Standardize Phenotype identifiers

Let us generate a `DataFrame` that stores a number of Phenotype identifiers, some of which corrupted:

In [ ]:
df_orig = pd.DataFrame(
    index=[
        "Specific learning disability"
        "Dystonia"
        "Cerebral hemorrhage"
        "Slurred speech"
        "This phenotype does not exist",
    ]
)
df_orig

We can check whether any of our values are validated against the ontology reference:

In [ ]:
validated = phenotypes.validate(df_orig.index, phenotypes.name)
df_orig.index[~validated]

## Ontology source versions

For any given entity, we can choose from a number of versions:

In [ ]:
lb.PublicSource.filter(entity="Phenotype").df()

When instantiating a Bionty object, we can choose a source or version:

In [ ]:
public_source = lb.PublicSource.filter(
    source="hp", version="2023-06-17", organism="human"
).one()
phenotypes= lb.Phenotype.public(public_source=public_source)
phenotypes

The currently used ontologies can be displayed using:

In [ ]:
lb.PublicSource.filter(currently_used=True).df()

In [ ]:
!lamin delete --force test-phenotype
!rm -r test-phenotype